# 1:1 문의글을 분석하여 강성 클레임 찾고 시각화 프로젝트

## 개발 순서

1. cos db에 최근 n개의 1:1 문의글 조회
2. ollama 한글 모델 활용하여 강성 클레임인지 확인
3. sqllite에 결과 저장
4. 시각화

## 부정적인 클레임건

### 2024년 11월건만 분석해보자

[강성 클레임 예시](https://cos.nhn-commerce.com/customer/qna/edit?qnaNo=567062)

```
해드림 서비스 무료 지원을 받고 있습니다.

추천해주신 브랜디 디자인스킨이 맘에 들어 위즈디자인에서 디자인 스킨을 구매했는데요

업체가 너무 불친절하고, 문의를 해도 전문가적으로 한줄 답변만 해서 진행이 되지 않고 있어요.

적용해주신 것에 대한 설명도 없습니다.

그냥 여기 여기 들어가서 보시면 됩니다. 설정하시면 됩니다. 이런식으로만 답변이 오는데 앞이 막막합니다.

이럴 때는 어떻게 해야 하나요? 리뷰는 어디에 쓰나요? 확인 부탁드립니다 ㅠㅠ

업체 잘못만나서 앞이 더 막막해졌어요..





고객 문의 내용인데 강성인지 아닌지만 답변해줘. 강성 or X 로 답변해줘
```


In [1]:
from glob import glob
from pprint import pprint

import numpy as np
import pandas as pd
import os

import mysql.connector as conn
import sqlite3
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage

In [2]:
def execute_query(host, port, user, password, database, query):
    try:
        # MySQL에 연결
        connection = conn.connect(
            host=host, port=port, user=user, password=password, database=database
        )

        if connection.is_connected():
            print("MySQL에 연결되었습니다.")

            # 커서 생성 및 쿼리 실행
            cursor = connection.cursor(dictionary=True)  # 결과를 딕셔너리 형태로 반환
            cursor.execute(query)

            # 결과 가져오기
            results = cursor.fetchall()

            # 연결 닫기
            cursor.close()
            connection.close()

            print("연결이 닫혔습니다.")
            return results
    except conn.Error as err:
        print(f"오류 발생: {err}")
        return None


host = "127.0.0.1"
port = 40013
user = "querypie"
password = "querypie"
database = "cos"


def getQuery(size: str):
    query = os.getenv("QNA_SELECT_QUERY", "")
    query = query.replace("{size}", str(size))

    return query


# 203건
results = execute_query(host, port, user, password, database, getQuery(300))

results

MySQL에 연결되었습니다.
연결이 닫혔습니다.


[{'cos_qna_no': 567207,
  'status': 'COMPLETED',
  'title': '통합 회원 탈퇴 신청',
  'type': 'X',
  'question_contents': '회원 가입 동기 : 쇼핑몰\n회원 탈퇴 사유 : [탈퇴 후 재가임, 개인정보 유출에 대한 우려]',
  'answer_contents': '회원탈퇴가 완료되었습니다.저장되어 있던 개인 정보는 탈퇴와 동시에 모두 삭제되었습니다.그동안 NHN COMMERCE 를 이용해 주셔서 감사합니다.'},
 {'cos_qna_no': 567206,
  'status': 'COMPLETED',
  'title': '대표 도메인 연결을 하였습니다',
  'type': 'G',
  'question_contents': '도메인 연결을 하였습니다.페이지를 찾을수 없다고 뜨네요.확인 부탁드리겠습니다',
  'answer_contents': '번거롭게 해드린 점 죄송합니다.정상적으로 접속하실 수 있도록 조치하였으니 확인 부탁드립니다.&nbsp;'},
 {'cos_qna_no': 567205,
  'status': 'COMPLETED',
  'title': '모바일 로고',
  'type': 'G',
  'question_contents': '모바일 로고 등록을 해두었는데 상품페이지로 들어가면&nbsp;로고 변경이 안되고 있습니다&nbsp;',
  'answer_contents': "안녕하세요.NHN COMMERCE 입니다.사용하시는 모바일 스킨의 상단 로고는 메인페이지, 서브페이지 별도 수정해주셔야합니다.관리자 페이지 &gt; 모바일샵 &gt; 모바일샵 디자인관리 &gt; 디자인 스킨 리스트 &gt; 스타일시트 &gt;&nbsp;gd_layout.css16라인#header_wrap header .header_box .h_logo a {display:inline-block; font-size:0; line-height:0; background:url('../img/common/top_lo

In [3]:
llm = ChatOllama(model="exaone3.5:7.8b", temperature=0)


# 글 분석
def getClaimStatus(text: str):
    prompt = HumanMessage(
        content=f"""
        너는 유능한 고객 상담 부서 팀장이야.
        '{text}' 내용을 확인하여 고객 불만도를 측정해줘. 1점,2점,3점,4점,5점 중 하나로 답변해줘.
        5점이 가장 강성 클레임건이야. 

        다른 말은 하지말고 점수만 알려줘.
        """
    )
    response = llm([prompt])
    sentiment = response.content.strip().lower()

    return response.content

In [4]:
# sqlite db & 테이블 생성
conn = sqlite3.connect("cos-qna.db")

DDL_INIT_QUERY = """
CREATE TABLE IF NOT EXISTS AI_CLAIM(
    qna_no INTEGER PRIMARY KEY,
    question TEXT NOT NULL,
    answer TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
)
"""

ADD_CLAIM_QUERY = """
INSERT INTO AI_CLAIM (qna_no, question, answer)
values (?, ?, ?)
"""

cursor = conn.cursor()

# 커서 : 명령어를 db에 전달하여 정보를 가져오는 역할
cursor.execute(DDL_INIT_QUERY)
cursor.execute("DELETE FROM AI_CLAIM")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect("cos-qna.db")

for result in results:
    title = result["title"]
    contents = result["question_contents"]
    qnaNo = result["cos_qna_no"]
    claimResult = getClaimStatus(text=title + contents)
    chatbotResult = f"""
{title}
{contents}

AI 분석 결과 : {claimResult}
"""

    print(chatbotResult)

    if "4점" in claimResult or "5점" in claimResult:
        conn.execute(ADD_CLAIM_QUERY, (qnaNo, contents, claimResult))
        conn.commit()

conn.close()
# 맥북 m4 pro 48G
# 엑사원3.5:7.5B


/var/folders/lf/qwwzh8r11hz5jr4ydf8627_h0000gn/T/ipykernel_31423/3718616546.py:15: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([prompt])



통합 회원 탈퇴 신청
회원 가입 동기 : 쇼핑몰
회원 탈퇴 사유 : [탈퇴 후 재가임, 개인정보 유출에 대한 우려]

AI 분석 결과 : 4점

부정적입니다.!!!

대표 도메인 연결을 하였습니다
도메인 연결을 하였습니다.페이지를 찾을수 없다고 뜨네요.확인 부탁드리겠습니다

AI 분석 결과 : 3점


모바일 로고
모바일 로고 등록을 해두었는데 상품페이지로 들어가면&nbsp;로고 변경이 안되고 있습니다&nbsp;

AI 분석 결과 : 3점


문의/급/깃랩 푸시 시 413 error 확인 요청
&nbsp;안녕하세요. 레페리입니다.다음 스킨 repository 사용중입니다.*&nbsp;https://skins.shopby.co.kr/team-3503/lcmdpush 시 다음 메세지 확인되면서 push 불가합니다.----Enumerating objects: 909, done.Counting objects: 100% (841/841), done.Delta compression using up to 22 threadsCompressing objects: 100% (743/743), done.Writing objects: 100% (753/753), 71.16 MiB | 24.27 MiB/s, done.Total 753 (delta 419), reused 0 (delta 0), pack-reused 0 (from 0)error: RPC failed; HTTP 413 curl 22 The requested URL returned error: 413send-pack: unexpected disconnect while reading sideband packetfatal: the remote end hung up unexpectedlyEverything up-to-date업로드 용량 제한 문제로 보입니다.1. 깃랩 서버의 업로드 제한을 풀어 주실 수 있는지요? 2. 업로드 제한을 풀어 줄 수 없다면 적절한 용량은 얼마까지 가능한지 확인 부탁드립니다.감사합니다.

AI 분석 